In [ ]:
// static/js/script.js

const COLORS = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'];

// --- 1. API Interaction ---

// 툴 목록 선택 시 타겟 툴 옵션 업데이트 (동기 함수)
function updateTargetToolOptions() {
    const toolSelect = document.getElementById("tool-select");
    const targetSelect = document.getElementById("target-tool-select");
    const selectedTools = Array.from(toolSelect.selectedOptions).map(o => o.value);
    const currentTarget = targetSelect.value;
    
    targetSelect.innerHTML = "";
    if (selectedTools.length === 0) {
        targetSelect.innerHTML = "<option disabled selected>Select tools above</option>";
        return;
    }
    selectedTools.forEach(tool => {
        const opt = document.createElement("option");
        opt.value = tool; opt.innerText = tool;
        targetSelect.appendChild(opt);
    });
    
    if (selectedTools.includes(currentTarget)) targetSelect.value = currentTarget;
    else targetSelect.selectedIndex = 0;
}

async function updateToolList() {
    const process = document.getElementById("process-select").value;
    const toolSelect = document.getElementById("tool-select");
    toolSelect.innerHTML = "<option>Loading...</option>";
    try {
        const res = await fetch(`/api/tools?process=${process}`);
        const data = await res.json();
        toolSelect.innerHTML = "";
        if(data.tools && data.tools.length) {
            data.tools.forEach(t => {
                const opt = document.createElement("option");
                opt.value = t; opt.innerText = t; toolSelect.appendChild(opt);
            });
        } else toolSelect.innerHTML = "<option disabled>No tools found</option>";
    } catch(e) { console.error(e); }
}

async function updateRecipeList() {
    const process = document.getElementById("process-select").value;
    const tools = Array.from(document.getElementById("tool-select").selectedOptions).map(o=>o.value);
    const startDate = document.getElementById("start-date").value;
    const endDate = document.getElementById("end-date").value;
    const recipeSelect = document.getElementById("recipe-select");
    
    if(!process || tools.length === 0) {
        alert("Select Process and Tools first!");
        return;
    }
    recipeSelect.innerHTML = "<option>Loading...</option>";
    
    try {
        const res = await fetch("/api/recipes", {
            method: "POST", headers: {"Content-Type":"application/json"},
            body: JSON.stringify({
                process: process, tools: tools,
                start_date: startDate, end_date: endDate
            })
        });
        const data = await res.json();
        recipeSelect.innerHTML = "";
        if(data.recipes && data.recipes.length) {
            data.recipes.forEach(r => {
                const opt = document.createElement("option");
                opt.value = r; opt.innerText = r; recipeSelect.appendChild(opt);
            });
        } else recipeSelect.innerHTML = "<option disabled>No recipes found</option>";
    } catch(e) { console.error(e); }
}

async function loadData() {
    const process = document.getElementById("process-select").value;
    const tools = Array.from(document.getElementById("tool-select").selectedOptions).map(o=>o.value);
    const recipes = Array.from(document.getElementById("recipe-select").selectedOptions).map(o=>o.value);
    const targetTool = document.getElementById("target-tool-select").value;

    if(!tools.length || !recipes.length) { alert("Select tools and recipes"); return; }
    
    const btn = document.querySelector(".btn-primary");
    btn.innerText = "Loading Files..."; btn.disabled = true;
    
    try {
        const res = await fetch("/api/load", {
            method: "POST", headers: {"Content-Type":"application/json"},
            body: JSON.stringify({
                process: process, tools: tools, recipes: recipes,
                start_date: document.getElementById("start-date").value,
                end_date: document.getElementById("end-date").value,
                target_tool: targetTool
            })
        });
        
        if(!res.ok) throw new Error((await res.json()).error);
        const treeData = await res.json();
        renderTree(treeData, targetTool);
        drawKG();

    } catch(e) { alert(e.message); }
    finally { btn.innerText = "Load Data (File I/O)"; btn.disabled = false; }
}

async function refreshView() {
    const issueType = document.getElementById("issue-select").value;
    const targetTool = document.getElementById("target-tool-select").value;

    try {
        const res = await fetch("/api/refresh_tree", {
            method: "POST", headers: {"Content-Type":"application/json"},
            body: JSON.stringify({ issue_type: issueType })
        });
        if(!res.ok) throw new Error((await res.json()).error);
        
        const treeData = await res.json();
        renderTree(treeData, targetTool);

    } catch(e) { console.error("Data not loaded yet or error:", e); }
}

async function drawKG() {
    const process = document.getElementById("process-select").value;
    const tools = Array.from(document.getElementById("tool-select").selectedOptions).map(o=>o.value);
    const recipes = Array.from(document.getElementById("recipe-select").selectedOptions).map(o=>o.value);
    const threshold = document.getElementById("kg-threshold").value;
    
    if(!process || !tools.length) return;

    try {
        const res = await fetch("/api/kg", {
            method: "POST", headers: {"Content-Type":"application/json"},
            body: JSON.stringify({ 
                process, tools, recipes, 
                start_date: document.getElementById("start-date").value, 
                end_date: document.getElementById("end-date").value,
                threshold: parseFloat(threshold) 
            })
        });
        if(res.ok) {
            const data = await res.json();
            renderForceGraph(data);
        }
    } catch(e) { console.error(e); }
}

// --- 2. Visualization Functions ---

function renderTree(treeData, targetTool) {
    d3.select("#tree-wrapper").selectAll("*").remove();
    document.getElementById("list-content").innerHTML = "<div style='color:#999; text-align:center; margin-top:20px;'>Select a node</div>";
    Plotly.purge('plotly-chart');
    
    const container = document.getElementById("tree-wrapper");
    // [중요] 높이가 0이 되지 않도록 CSS에서 min-height 설정하거나 여기서 고정값 체크
    const width = container.clientWidth || 400;
    const height = container.clientHeight || 300;
    
    const svg = d3.select("#tree-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(d3.zoom().on("zoom", (e) => g.attr("transform", e.transform)))
        .on("dblclick.zoom", null);
    const g = svg.append("g");
    
    const root = d3.hierarchy(treeData, d => d.children);
    const tree = d3.tree().nodeSize([40, 140]);
    const nodes = tree(root);

    const link = g.selectAll(".link").data(nodes.links()).enter().append("path")
        .attr("class", "link").attr("d", d3.linkHorizontal().x(d => d.y).y(d => d.x));

    const node = g.selectAll(".node").data(nodes.descendants()).enter().append("g")
        .attr("class", "node").attr("transform", d => `translate(${d.y},${d.x})`);

    node.append("circle").attr("r", 8)
        .on("click", async function(event, d) {
            d3.selectAll("circle").classed("selected", false);
            d3.select(this).classed("selected", true);
            const ancestors = new Set(d.ancestors());
            link.classed("highlight", l => ancestors.has(l.target)).classed("dimmed", l => !ancestors.has(l.target));

            // Chart Fetch
            if (!d.data.chart_config) {
                document.getElementById("placeholder").innerText = "Fetching Chart...";
                document.getElementById("placeholder").style.display = "block";
                Plotly.purge('plotly-chart');
                try {
                    const res = await fetch("/api/chart", {
                        method: "POST", headers: {"Content-Type": "application/json"},
                        body: JSON.stringify({ sensor_name: d.data.name })
                    });
                    if(res.ok) d.data.chart_config = await res.json();
                } catch(e) { console.error(e); }
            }
            if (d.data.chart_config) {
                document.getElementById("placeholder").style.display = "none";
                Plotly.newPlot('plotly-chart', d.data.chart_config.data, d.data.chart_config.layout, {responsive: true});
            }

            // List
            const listContent = document.getElementById("list-content");
            const listHeader = document.getElementById("list-header");
            listContent.innerHTML = "";
            if (d.data.related_sensors && d.data.related_sensors.length > 0) {
                listHeader.innerHTML = `${d.data.name}<br><small style='color:#d32f2f;'>Diff Sorted</small>`;
                d.data.related_sensors.forEach(sensor => {
                    const item = document.createElement("div");
                    item.className = "sensor-item";
                    const diffVal = sensor.diff_value ? Math.round(sensor.diff_value * 10)/10 : 0;
                    item.innerHTML = `<span>${sensor.name}</span><span style="font-size:0.8em">Diff: ${diffVal}</span>`;
                    
                    item.onclick = async () => {
                         // 연관 센서도 차트 데이터가 없으면 요청해야 함 (여기선 생략하거나 동일하게 fetch 구현)
                         // 간단하게 하려면 메인 센서만 차트 지원
                    };
                    listContent.appendChild(item);
                });
            } else {
                listHeader.innerText = "Related Sensors";
                listContent.innerHTML = "<div style='color:#999; text-align:center; margin-top:20px;'>No data</div>";
            }
            event.stopPropagation();
        });

    node.append("text").attr("dy", ".35em").attr("x", d => d.children ? -12 : 12)
        .style("text-anchor", d => d.children ? "end" : "start").text(d => d.data.name);

    svg.call(d3.zoom().transform, d3.zoomIdentity.translate(40, height / 2));
}

function renderForceGraph(graphData) {
    d3.select("#kg-wrapper").selectAll("*").remove();
    const container = document.getElementById("kg-wrapper");
    const width = container.clientWidth || 400;
    const height = container.clientHeight || 300;

    const svg = d3.select("#kg-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(d3.zoom().on("zoom", (e) => g.attr("transform", e.transform)));
    const g = svg.append("g");

    const simulation = d3.forceSimulation(graphData.nodes)
        .force("link", d3.forceLink(graphData.links).id(d => d.id).distance(100))
        .force("charge", d3.forceManyBody().strength(-300))
        .force("center", d3.forceCenter(width / 2, height / 2))
        .force("collide", d3.forceCollide().radius(30));

    const link = g.append("g").selectAll("line").data(graphData.links).enter().append("line")
        .attr("class", d => d.type === 'structure' ? 'link-structure' : 'link-correlation');

    const node = g.append("g").selectAll("circle").data(graphData.nodes).enter().append("circle")
        .attr("r", d => d.radius).attr("class", d => d.group === 'group_node' ? 'node-group' : 'node-sensor')
        .call(d3.drag().on("start", dragstart).on("drag", dragged).on("end", dragend));

    const label = g.append("g").selectAll("text").data(graphData.nodes).enter().append("text")
        .attr("class", "label").attr("dy", -15).attr("text-anchor", "middle").text(d => d.id);

    simulation.on("tick", () => {
        link.attr("x1", d => d.source.x).attr("y1", d => d.source.y).attr("x2", d => d.target.x).attr("y2", d => d.target.y);
        node.attr("cx", d => d.x).attr("cy", d => d.y);
        label.attr("x", d => d.x).attr("y", d => d.y);
    });

    function dragstart(event, d) { if (!event.active) simulation.alphaTarget(0.3).restart(); d.fx = d.x; d.fy = d.y; }
    function dragged(event, d) { d.fx = event.x; d.fy = event.y; }
    function dragend(event, d) { if (!event.active) simulation.alphaTarget(0); d.fx = null; d.fy = null; }
}

In [ ]:
Ctrl + Shift + R

In [ ]:
# kg_logic.py

# ... 기존 import 및 함수들 ...

def generate_correlation_matrix(process, tools, recipes, start_date_str, end_date_str):
    """
    선택된 조건의 데이터로 상관계수 매트릭스(z, x, y)를 생성하여 반환
    """
    try:
        s_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
        e_date = datetime.strptime(end_date_str, "%Y-%m-%d").date()
    except ValueError: return None

    # 1. 파일 수집 (generate_kg_data와 동일 로직, 중복 방지를 위해 함수 분리 추천하지만 여기선 그대로 작성)
    files_to_read = []
    for tool in tools:
        tool_dir = os.path.join(STORAGE_ROOT, process, tool)
        if not os.path.exists(tool_dir): continue
        for f in os.listdir(tool_dir):
            if f.endswith(".parquet"):
                f_tool, f_date, f_recipe = parse_filename(f)
                if f_tool and f_date and s_date <= f_date <= e_date and f_recipe in recipes:
                    files_to_read.append(os.path.join(tool_dir, f))

    if not files_to_read: return None

    # 2. 데이터 통합
    sensor_history_map = {}
    for file_path in files_to_read:
        try:
            df = pq.read_table(file_path).to_pandas()
            for _, row in df.iterrows():
                s_name = row['sensor']
                hist = list(row['history'])
                if s_name not in sensor_history_map: sensor_history_map[s_name] = []
                sensor_history_map[s_name].extend(hist)
        except: continue

    if not sensor_history_map: return None

    # 3. 상관관계 매트릭스 계산
    ts_df = pd.DataFrame(sensor_history_map)
    # NaN 결측치 제거 (상관계수 계산 오류 방지)
    ts_df = ts_df.dropna(axis=1) 
    corr_df = ts_df.corr()

    # Plotly Heatmap 포맷으로 반환
    return {
        "z": corr_df.values.tolist(),      # 값 (2D 배열)
        "x": corr_df.columns.tolist(),     # X축 라벨 (센서 이름)
        "y": corr_df.index.tolist()        # Y축 라벨 (센서 이름)
    }

In [ ]:
# main.py

# ... 기존 코드 ...

@app.post("/api/correlation_matrix")
async def get_corr_matrix(req: KGRequest): # KGRequest 모델 재사용 (threshold는 안 쓰지만 무관)
    import kg_logic
    data = kg_logic.generate_correlation_matrix(
        req.process, req.tools, req.recipes, req.start_date, req.end_date
    )
    if not data:
        return JSONResponse({"error": "No correlation data found"}, status_code=404)
    return JSONResponse(data)

In [ ]:
<!-- templates/index.html -->

<!-- ... 우측 하단 영역 수정 ... -->
<div class="right-bottom">
    <div style="padding:10px 15px; background:#fafafa; border-bottom:1px solid #eee; display:flex; justify-content:space-between; align-items:center;">
        <span style="font-weight:bold; font-size:0.95rem;">Correlation Analysis</span>
        
        <!-- [신규] 뷰 전환 스위치 -->
        <div style="display:flex; gap:10px; font-size:0.85rem;">
            <label style="cursor:pointer;">
                <input type="radio" name="kg-view" value="graph" checked onclick="switchKGView('graph')"> Graph
            </label>
            <label style="cursor:pointer;">
                <input type="radio" name="kg-view" value="heatmap" onclick="switchKGView('heatmap')"> Heatmap
            </label>
        </div>
    </div>
    
    <!-- 내용물이 들어갈 곳 -->
    <div id="kg-wrapper" style="flex:1; position:relative;"></div>
</div>

In [ ]:
// static/js/script.js

// [신규] 뷰 전환 로직
let currentKGView = 'graph'; // 'graph' or 'heatmap'

function switchKGView(viewType) {
    currentKGView = viewType;
    // 데이터가 이미 로드된 상태라면 즉시 다시 그리기
    drawKGAnalysis(); 
}

// [변경] 기존 drawKG -> drawKGAnalysis로 통합
async function drawKGAnalysis() {
    const process = document.getElementById("process-select").value;
    const tools = Array.from(document.getElementById("tool-select").selectedOptions).map(o=>o.value);
    const recipes = Array.from(document.getElementById("recipe-select").selectedOptions).map(o=>o.value);
    const threshold = document.getElementById("kg-threshold").value;
    
    // 필수 선택값 체크
    if(!process || !tools.length || !recipes.length) return;

    // 1. D3 Graph 그리기
    if (currentKGView === 'graph') {
        try {
            const res = await fetch("/api/kg", {
                method: "POST", headers: {"Content-Type":"application/json"},
                body: JSON.stringify({ 
                    process, tools, recipes, 
                    start_date: document.getElementById("start-date").value, 
                    end_date: document.getElementById("end-date").value,
                    threshold: parseFloat(threshold) 
                })
            });
            if(res.ok) {
                const data = await res.json();
                // Plotly 흔적이 있으면 지우기
                Plotly.purge('kg-wrapper');
                // D3 렌더링
                renderForceGraph(data);
            }
        } catch(e) { console.error(e); }
    } 
    // 2. Plotly Heatmap 그리기
    else if (currentKGView === 'heatmap') {
        try {
            // D3 SVG 지우기
            d3.select("#kg-wrapper").selectAll("*").remove();
            // 로딩 표시
            document.getElementById("kg-wrapper").innerHTML = "<div style='text-align:center; padding-top:20px;'>Loading Matrix...</div>";

            const res = await fetch("/api/correlation_matrix", {
                method: "POST", headers: {"Content-Type":"application/json"},
                body: JSON.stringify({ 
                    process, tools, recipes, 
                    start_date: document.getElementById("start-date").value, 
                    end_date: document.getElementById("end-date").value,
                    threshold: 0 // 히트맵은 전체 다 보여주므로 threshold 무관
                })
            });
            
            if(res.ok) {
                const data = await res.json();
                document.getElementById("kg-wrapper").innerHTML = ""; // 로딩 제거
                drawPlotlyHeatmap(data);
            }
        } catch(e) { console.error(e); }
    }
}

// [신규] 히트맵 렌더링 함수
function drawPlotlyHeatmap(data) {
    const trace = {
        z: data.z,
        x: data.x,
        y: data.y,
        type: 'heatmap',
        colorscale: 'RdBu', // Red(양의상관) - White - Blue(음의상관)
        zmin: -1,
        zmax: 1,
        hoverongaps: false
    };

    const layout = {
        margin: {t: 30, r: 30, b: 50, l: 50}, // 여백 자동 조정
        paper_bgcolor: 'rgba(0,0,0,0)',
        plot_bgcolor: 'rgba(0,0,0,0)',
        xaxis: { ticks: '', side: 'bottom' },
        yaxis: { ticks: '', ticks: '' , autorange: 'reversed' } // Y축 뒤집기 (상단이 0번 인덱스)
    };
    
    const config = {responsive: true, displayModeBar: false};

    Plotly.newPlot('kg-wrapper', [trace], layout, config);
}

// [변경] loadData 함수 마지막에 호출 부분 수정
async function loadData() {
    // ... (기존 로직 동일) ...
    try {
        // ... 파일 로딩 및 트리 렌더링 ...
        renderTree(treeData, targetTool);

        // [변경] 통합 함수 호출
        drawKGAnalysis(); 

    } catch(e) { alert(e.message); }
    // ...
}